# **-----------------------**>**AI MUSIC GENERATION PROJECT**<**-----------------------**

# Install Required Libraries

In [1]:
# Terminal
#pip install music21 keras tensorflow mido

import warnings
import time

# Ignore warnings
warnings.filterwarnings("ignore")

# Load and Preprocess Data

In [2]:
import os
import glob
import numpy as np
from music21 import converter, instrument, note, chord, stream

# Set data path
data_path = r"D:\# DATA SCIENCE\# PROJECTS\- PROJECTS INTERNSHIPS\CODEALPHA - AI ENGINEERING\Music Generation with AI Project\Data"

def get_notes(data_path):
    notes = []
    
    for folder in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder)
        if os.path.isdir(folder_path):
            for file in glob.glob(os.path.join(folder_path, "*.mid")):
                midi = converter.parse(file)

                notes_to_parse = None
                parts = instrument.partitionByInstrument(midi)
                
                if parts:  # File has instrument parts
                    notes_to_parse = parts.parts[0].recurse()
                else:  # File has flat notes
                    notes_to_parse = midi.flat.notes
                
                for element in notes_to_parse:
                    if isinstance(element, note.Note):
                        notes.append(str(element.pitch))
                    elif isinstance(element, chord.Chord):
                        notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

# Get all notes from the dataset
notes = get_notes(data_path)
print(f"Number of notes extracted: {len(notes)}")

Number of notes extracted: 11362


# Prepare Data for the Model

In [3]:
# Prepare Data for the Model
from tensorflow.keras.utils import to_categorical

# Get all unique notes
unique_notes = list(set(notes))
n_vocab = len(unique_notes)

# Map notes to integers
note_to_int = {note: i for i, note in enumerate(unique_notes)}
int_notes = [note_to_int[note] for note in notes]

# Prepare sequences of notes for training
sequence_length = 100
network_input = []
network_output = []

for i in range(0, len(int_notes) - sequence_length):
    seq_in = int_notes[i:i + sequence_length]
    seq_out = int_notes[i + sequence_length]
    
    network_input.append(seq_in)
    network_output.append(seq_out)

n_patterns = len(network_input)

# Reshape for LSTM model (RNN)
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)

# One-hot encode the output
network_output = to_categorical(network_output)

# Build the LSTM Model

In [4]:
# Build the LSTM Model
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.optimizers import Adam

# Build the LSTM model
model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 lstm_1 (LSTM)               (None, 512)               2099200   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0

# Train the Model

In [5]:
# Train the Model
model.fit(network_input, network_output, epochs=10, batch_size=64)  

Epoch 1/10
176/176 [==============================] - 298s 2s/step - loss: 4.8576
Epoch 2/10
176/176 [==============================] - 443s 3s/step - loss: 4.7862
Epoch 3/10
176/176 [==============================] - 451s 3s/step - loss: 4.8013
Epoch 4/10
176/176 [==============================] - 449s 3s/step - loss: 4.7598
Epoch 5/10
176/176 [==============================] - 445s 3s/step - loss: 4.7495
Epoch 6/10
176/176 [==============================] - 448s 3s/step - loss: 4.7461
Epoch 7/10
176/176 [==============================] - 446s 3s/step - loss: 4.7441
Epoch 8/10
176/176 [==============================] - 441s 3s/step - loss: 4.7409
Epoch 9/10
176/176 [==============================] - 447s 3s/step - loss: 4.7396
Epoch 10/10
176/176 [==============================] - 442s 3s/step - loss: 4.7411


# Generate Music

In [6]:
# Generate Music
import random

def generate_music(model, network_input, int_to_note, n_vocab, output_length=500, temperature=1.0):
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]
    prediction_output = []

    # Generate output_length notes
    for _ in range(output_length):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        prediction = np.log(prediction + 1e-6) / temperature
        prediction = np.exp(prediction)
        prediction = prediction / np.sum(prediction)
        index = np.random.choice(range(n_vocab), p=prediction[0])
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:]

    return prediction_output

# Map integers back to notes
int_to_note = {i: note for note, i in note_to_int.items()}

# Generate and save multiple pieces of music
def create_midi(prediction_output, output_file):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                note_obj = note.Note(int(current_note))
                note_obj.storedInstrument = instrument.Piano()
                notes.append(note_obj)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += np.random.uniform(0.5, 1.0)  # Randomize offset

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)

for i in range(5):  # Generate 5 different pieces
    predicted_notes = generate_music(model, network_input, int_to_note, n_vocab, output_length=500, temperature=1.0)
    output_file = f'test_output_{i}.mid'
    create_midi(predicted_notes, output_file)
    print(f"MIDI file {output_file} saved.")


MIDI file test_output_0.mid saved.
MIDI file test_output_1.mid saved.
MIDI file test_output_2.mid saved.
MIDI file test_output_3.mid saved.
MIDI file test_output_4.mid saved.


# Save the Trained Model

In [7]:
# Save the model
model.save('music_generation_model.h5')


Load the saved model

In [8]:
from keras.models import load_model

# Load the saved model
model = load_model('music_generation_model.h5')


# Load & Generate a New Song

In [11]:
import time
import os

# Path to the folder where you want to save the generated music files
save_path = r'D:\# DATA SCIENCE\# PROJECTS\- PROJECTS INTERNSHIPS\CODEALPHA - AI ENGINEERING\Music Generation with AI Project\Generated Music'

# Ensure the folder exists, create it if it doesn't
os.makedirs(save_path, exist_ok=True)

# Regenerate a new piece of music
predicted_notes = generate_music(model, network_input, int_to_note, n_vocab, output_length=500, temperature=1.0)

# Generate a unique filename using the current timestamp
output_file = os.path.join(save_path, f'new_song_{time.strftime("%Y%m%d_%H%M%S")}.mid')

# Save the MIDI file with the unique filename in the specified folder
create_midi(predicted_notes, output_file)
print(f"New MIDI file {output_file} saved to {save_path}.")


New MIDI file D:\# DATA SCIENCE\# PROJECTS\- PROJECTS INTERNSHIPS\CODEALPHA - AI ENGINEERING\Music Generation with AI Project\Generated Music\new_song_20240926_044925.mid saved to D:\# DATA SCIENCE\# PROJECTS\- PROJECTS INTERNSHIPS\CODEALPHA - AI ENGINEERING\Music Generation with AI Project\Generated Music.
